In [1]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
val researchYear = "2000"

In [3]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [4]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [5]:
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._


In [6]:
df.filter(df("mime") === "text/html").select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                    2819034|
+---------------------------+



In [7]:
df.select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                    3565614|
+---------------------------+



In [8]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").orderBy(desc("URLs")).show(300,false)

+-----------------------------+-------+
|mime                         |URLs   |
+-----------------------------+-------+
|text/html                    |2819034|
|im                           |433672 |
|alexa/dat                    |162034 |
|image/gif                    |86455  |
|image/jpeg                   |77716  |
|unknown                      |44559  |
|text/pl                      |34792  |
|text/plain                   |27483  |
|unk                          |27214  |
|application/zip              |1030   |
|audio/x-pn-realaudio         |698    |
|text/css                     |660    |
|application/postscript       |605    |
|application/octet-stream     |524    |
|application/x-javascript     |467    |
|application/pdf              |367    |
|image/png                    |345    |
|application/x-zip-compressed |337    |
|audio/midi                   |322    |
|audio/mpeg                   |283    |
|application/msword           |150    |
|audio/basic                  |145    |


In [10]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").count()

109